<a href="https://colab.research.google.com/github/pavannanaiah/movies/blob/main/Moveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Collection


In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from numpy import NAN

# Basic Data Collection

url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2021-01-01,2023-08-3&count=50"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    movie_headers = soup.find_all('div', class_='lister-item mode-advanced')

    movie_ids = []
    movie_names = []
    movie_genres = []
    movie_years = []
    movie_ratings = []
    movie_durations = []
    movie_votes = []

    for header in movie_headers:

        href = header.find('a')['href']
        movie_id = href.split('/')[2] if href.startswith('/title/') else ''

        movie_name = header.find('h3', class_='lister-item-header').find('a').text.strip()
        movie_genre = header.find('p', class_='text-muted').find('span', class_='genre').text.strip()
        movie_year = header.find('span', class_='lister-item-year').text.strip('()')

        rating_text = header.find('div', class_="inline-block ratings-imdb-rating", attrs={"name":"ir"})
        movie_rating = ''#rating_text['data-value'] if vote_text else ''

        movie_duration = header.find('p', class_='text-muted').find('span', class_='runtime').text.strip()

        vote_text = header.find('span', attrs={"name":"nv"})
        movie_vote = vote_text['data-value'] if vote_text else ''

        movie_ids.append(movie_id)
        movie_names.append(movie_name)
        movie_genres.append(movie_genre)
        movie_years.append(movie_year)
        movie_ratings.append(movie_rating)
        movie_durations.append(movie_duration)
        movie_votes.append(movie_vote)


    movie_data = pd.DataFrame({'Movie ID':movie_ids,'Movie Name': movie_names,
                               'Movie Genre': movie_genres, 'Movie Year': movie_years,
                               'Movie Rating':movie_ratings,'Movie Duration': movie_durations,
                               'Movie Votes': movie_votes})

    print(movie_data)

else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)


# Collect Budget and Revenue Data






      Movie ID                                     Movie Name  \
0    tt1517268                                         Barbie   
1   tt15398776                                    Oppenheimer   
2   tt10638522                                     Talk to Me   
3   tt15789038                                      Elemental   
4    tt4495098                                   Gran Turismo   
5    tt5537002                     Killers of the Flower Moon   
6   tt13651794                                   May December   
7   tt17527468                                        Bottoms   
8    tt8589698    Teenage Mutant Ninja Turtles: Mutant Mayhem   
9    tt5971474                             The Little Mermaid   
10   tt4873118                                   The Covenant   
11  tt11040844                                    The Machine   
12  tt15671028                               No Hard Feelings   
13  tt10366206                           John Wick: Chapter 4   
14  tt13560574           

In [57]:

# Collect Budget , Revenue Data and Language

movie_languages = []
movie_revenues = []
movie_budgets = []


for index, movie in movie_data.iterrows():
  url = "https://www.imdb.com/title/" + movie["Movie ID"]
  #print(url)

  HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

  response = requests.get(url, headers= HEADERS)

  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    money_element = soup.find("div", {"data-testid": "title-boxoffice-section"})
    languages = soup.find("li", {"data-testid": "title-details-languages"})
    #print (languages)
    if (money_element != None):
      #For budget
      if money_element.find("li", {"data-testid": "title-boxoffice-budget"}) != None:
        budget = money_element.find("li", {"data-testid": "title-boxoffice-budget"}).text.strip()
      else:
        budget = " "
      #print(budget)

     #For revenue
      if money_element.find("li", {"data-testid": "title-boxoffice-cumulativeworldwidegross"}) != None:
        revenue =  money_element.find("li", {"data-testid": "title-boxoffice-cumulativeworldwidegross"}).text.strip()
      else:
        revenue = " "

      #For langauges
      language_links = languages.find_all("a", {"class": "ipc-metadata-list-item__list-content-item--link"})
      languages = ', '.join([link.text.strip() for link in language_links])

      movie_budget = budget
      movie_revenue = revenue
      movie_language = languages
    else:
      movie_budget = " "
      movie_revenue = " "

    movie_budgets.append(movie_budget.replace("Budget","").replace(" (estimated)","").replace("$",""))
    movie_revenues.append(movie_revenue.replace("Gross worldwide","").replace("$",""))
    movie_languages.append(languages)

  else:
    movie_budgets.append(" ")
    movie_revenues.append(" ")
    movie_languages.append(" ")


movie_data['Movie Budget'] = movie_budgets
movie_data['Movie Revenue'] = movie_revenues
movie_data['Movie Languages'] = movie_languages

print(movie_data)



      Movie ID                                     Movie Name  \
0    tt1517268                                         Barbie   
1   tt15398776                                    Oppenheimer   
2   tt10638522                                     Talk to Me   
3   tt15789038                                      Elemental   
4    tt4495098                                   Gran Turismo   
5    tt5537002                     Killers of the Flower Moon   
6   tt13651794                                   May December   
7   tt17527468                                        Bottoms   
8    tt8589698    Teenage Mutant Ninja Turtles: Mutant Mayhem   
9    tt5971474                             The Little Mermaid   
10   tt4873118                                   The Covenant   
11  tt11040844                                    The Machine   
12  tt15671028                               No Hard Feelings   
13  tt10366206                           John Wick: Chapter 4   
14  tt13560574           